In [25]:
from imports import pd, time
import numpy as np
import yfinance as source

In [35]:
# URLs for the NASDAQ Trader Symbol Directory files
# List is updated everyday.
nasdaq_url = "ftp://ftp.nasdaqtrader.com/SymbolDirectory/nasdaqlisted.txt"
# other_url = "ftp://ftp.nasdaqtrader.com/SymbolDirectory/otherlisted.txt"

# Load NASDAQ-listed stocks
nasdaq_data = pd.read_csv(nasdaq_url, sep="|")
# Remove the last row (footer info)
nasdaq_data = nasdaq_data[:-1]

# # Load other-listed stocks (including NYSE, AMEX, etc.)
# other_data = pd.read_csv(other_url, sep="|")
# # Filter for NYSE stocks
# nyse_stocks = other_data[other_data["Exchange"] == "N"]

In [36]:
#  Filter for companies that are not bankrupt, delisted, etc.
normal_cmps = nasdaq_data.where(nasdaq_data["Financial Status"] == "N").dropna()
normal_cmps.index = range(len(normal_cmps))
normal_cmps

,Symbol,Security Name,Market Category,Test Issue,Financial Status,Round Lot Size,ETF,NextShares
0,AADI,"Aadi Bioscience, Inc. - Common Stock",S,N,N,100.0,N,N
1,AADR,AdvisorShares Dorsey Wright ADR ETF,G,N,N,100.0,Y,N
2,AAL,"American Airlines Group, Inc. - Common Stock",Q,N,N,100.0,N,N
3,AAME,Atlantic American Corporation - Common Stock,G,N,N,100.0,N,N
4,AAOI,"Applied Optoelectronics, Inc. - Common Stock",G,N,N,100.0,N,N
...,...,...,...,...,...,...,...,...
4306,ZVSA,"ZyVersa Therapeutics, Inc. - Common Stock",S,N,N,100.0,N,N
4307,ZYBT,Zhengye Biotechnology Holding Limited - Ordina...,S,N,N,100.0,N,N
4308,ZYME,Zymeworks Inc. - Common Stock,Q,N,N,100.0,N,N
4309,ZYXI,"Zynex, Inc. - Common Stock",Q,N,N,100.0,N,N


In [37]:
nasdaq_tickers = normal_cmps[["Symbol", "Security Name"]]
nasdaq_tickers.loc[:, "Exchange"] = "NASDAQ"
nasdaq_tickers

/var/folders/k2/yqlt87156k17x32rmffq7jfr0000gn/T/ipykernel_15356/860605534.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nasdaq_tickers.loc[:, "Exchange"] = "NASDAQ"


,Symbol,Security Name,Exchange
0,AADI,"Aadi Bioscience, Inc. - Common Stock",NASDAQ
1,AADR,AdvisorShares Dorsey Wright ADR ETF,NASDAQ
2,AAL,"American Airlines Group, Inc. - Common Stock",NASDAQ
3,AAME,Atlantic American Corporation - Common Stock,NASDAQ
4,AAOI,"Applied Optoelectronics, Inc. - Common Stock",NASDAQ
...,...,...,...
4306,ZVSA,"ZyVersa Therapeutics, Inc. - Common Stock",NASDAQ
4307,ZYBT,Zhengye Biotechnology Holding Limited - Ordina...,NASDAQ
4308,ZYME,Zymeworks Inc. - Common Stock,NASDAQ
4309,ZYXI,"Zynex, Inc. - Common Stock",NASDAQ


In [41]:
data_points = ['symbol', 'shortName', 'currency', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose',
               'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield',
              'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'forwardPE', 'volume',  'marketCap',
              'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 
              'twoHundredDayAverage', 'profitMargins', 'bookValue', 'priceToBook', 'earningsQuarterlyGrowth', 
              'netIncomeToCommon', 'trailingEps', 'forwardEps', 'enterpriseToRevenue', 'enterpriseToEbitda', '52WeekChange',
              'ebitda', 'totalDebt', 'quickRatio', 'currentRatio', 'totalRevenue', 'debtToEquity', 'revenuePerShare', 'returnOnAssets', 
              'returnOnEquity', 'grossProfits', 'freeCashflow', 'operatingCashflow', 'earningsGrowth', 'revenueGrowth', 'grossMargins', 
              'ebitdaMargins', 'operatingMargins', 'trailingPegRatio']

data = []
def download_data(stocks):
    stk = source.Tickers(" ".join(stocks))
    for ticker in stocks:
        try:
            stock_info = stk.tickers[ticker].info
            row = {key: stock_info.get(key, None) for key in data_points}
            data.append(row)
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")


In [43]:
batch_size = 25
m = len(list(nasdaq_tickers["Symbol"]))

# Running below commented lines will fetch data for all the stocks in the NASDAQ list and uncomment if fetching
# data for first time.
for idx in range(0, m, batch_size):
    download_data(list(nasdaq_tickers["Symbol"][idx:idx+batch_size]))
    time.sleep(5)
print(f"Done fetching data for {len(data)} stocks.")

Error fetching data for ATLN: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error fetching data for ATLO: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error fetching data for ATYR: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error fetching data for AUBN: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LUCYW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LUCYW&crumb=EywNrLHzlza


Done fetching data for 5712 stocks.


In [83]:
stocksData = [i for i in data if i!=None]
print(f"{len(stocksData)} stocks data fetched!")


5712 stocks data fetched!


***Now, we have to standarize the data for some of the columns and remove duplicates***

In [54]:
stocksData_df = pd.DataFrame(stocksData, columns=data_points)
stocksData_df = stocksData_df.dropna(subset=["symbol"])
stocksData_df = stocksData_df.replace([np.nan], 0.0)
stocksData_df.columns = [i.capitalize() for i in stocksData_df.columns]
stocksData_df['Marketcap'] = stocksData_df['Marketcap']/(10e8) # Converting marketcap in order of 100 million
stocksData_df['Returnonequity'] = stocksData_df['Returnonequity']*100 # Converting to percentage
stocksData_df['Earningsgrowth'] = stocksData_df['Earningsgrowth']*100 # Converting to percentage
stocksData_df['Revenuegrowth'] = stocksData_df['Revenuegrowth']*100 # Converting to percentage
stocksData_df['Profitmargins'] = stocksData_df['Profitmargins']*100 # Converting to percentage
stocksData_df['Ebitdamargins'] = stocksData_df['Ebitdamargins']*100 # Converting to percentage
stocksData_df['Totalrevenue'] = stocksData_df['Totalrevenue']/(10e8) # Converting total revenue in order of 100 million
stocksData_df['Grossprofits'] = stocksData_df['Grossprofits']/(10e8) # Converting gross profits in order of 100 million
stocksData_df['Freecashflow'] = stocksData_df['Freecashflow']/(10e8) # Converting free cash flow in order of 100 million
stocksData_df['Operatingcashflow'] = stocksData_df['Operatingcashflow']/(10e8) # Converting free cash flow in order of 100 million
stocksData_df = stocksData_df[stocksData_df['Currency']!='0.0']
stocksData_df = stocksData_df.round(2)
# Drop duplicates based on the Symbol column, keeping the first occurrence
stocksData_df = stocksData_df.drop_duplicates(subset="Symbol", keep="first")
stocksData_df.reset_index(drop=True, inplace=True)

stocksData_df.to_csv("stocksData.csv", index=False)
print("Saved data to csv file.")

Saved data to csv file.


In [116]:
df = pd.read_csv("stocksData.csv")
df.head(n=10)

,Symbol,Shortname,Currency,Previousclose,Open,Daylow,Dayhigh,Regularmarketpreviousclose,Regularmarketopen,Regularmarketdaylow,...,Returnonequity,Grossprofits,Freecashflow,Operatingcashflow,Earningsgrowth,Revenuegrowth,Grossmargins,Ebitdamargins,Operatingmargins,Trailingpegratio
0,AADI,"Aadi Bioscience, Inc.",USD,3.14,3.10,3.00,3.10,3.14,3.10,3.00,...,-65.95,-0.03,-0.04,-5.414100e+07,0.0,21.0,-1.10,-252.87,-1.49,0.00
1,AADR,AdvisorShares Dorsey Wright,USD,74.51,72.97,74.25,74.51,74.51,72.97,74.25,...,0.00,0.00,0.00,0.000000e+00,0.0,0.0,0.00,0.00,0.00,0.00
2,AAL,"American Airlines Group, Inc.",USD,16.96,16.80,16.70,17.54,16.96,16.80,16.70,...,0.00,12.89,0.45,2.234000e+09,0.0,1.2,0.24,9.45,0.05,0.21
3,AAME,Atlantic American Corporation,USD,1.51,1.44,1.44,1.55,1.51,1.44,1.44,...,-3.09,0.01,0.01,2.615000e+06,0.0,-3.1,0.08,-0.19,0.00,0.00
4,AAOI,"Applied Optoelectronics, Inc.",USD,30.69,28.58,25.80,29.72,30.69,28.58,25.80,...,-41.44,0.05,-0.04,-4.336000e+07,0.0,4.2,0.26,-23.26,-0.25,0.00
5,AAON,"AAON, Inc.",USD,132.71,128.78,114.67,129.38,132.71,128.78,114.67,...,25.81,0.43,0.09,2.434370e+08,-68.6,16.8,0.36,25.69,0.04,0.00
6,AAPB,GraniteShares 2x Long AAPL Dail,USD,26.48,26.79,26.67,28.24,26.48,26.79,26.67,...,0.00,0.00,0.00,0.000000e+00,0.0,0.0,0.00,0.00,0.00,0.00
7,AAPD,Direxion Daily AAPL Bear 1X Sha,USD,17.17,17.04,16.58,17.05,17.17,17.04,16.58,...,0.00,0.00,0.00,0.000000e+00,0.0,0.0,0.00,0.00,0.00,0.00
8,AAPG,0.0,USD,17.38,17.39,17.00,17.50,17.38,17.39,17.00,...,0.00,0.00,0.00,0.000000e+00,0.0,0.0,0.00,0.00,0.00,0.00
9,AAPL,Apple Inc.,USD,222.78,224.03,224.00,230.48,222.78,224.03,224.00,...,157.41,180.68,110.85,1.182540e+11,-34.1,6.1,0.46,34.44,0.31,2.03


In [112]:
for col in df.columns:
    print("Column:", col, df[col].idxmax(), df.loc[df[col].idxmax()]['Symbol'], df[col].idxmin(), df.loc[df[col].idxmin()]['Symbol'])

Column: Symbol 5709 ZZZ 0 AADI
Column: Shortname 5697 ZSPC 8 AAPG
Column: Currency 0 AADI 1058 DFGP
Column: Previousclose 497 BKNG 1058 DFGP
Column: Open 497 BKNG 89 AFJKU
Column: Daylow 497 BKNG 1058 DFGP
Column: Dayhigh 497 BKNG 1058 DFGP
Column: Regularmarketpreviousclose 497 BKNG 1058 DFGP
Column: Regularmarketopen 497 BKNG 89 AFJKU
Column: Regularmarketdaylow 497 BKNG 1058 DFGP
Column: Regularmarketdayhigh 497 BKNG 1058 DFGP
Column: Dividendrate 497 BKNG 0 AADI
Column: Dividendyield 3302 ICON 0 AADI
Column: Fiveyearavgdividendyield 3613 LEE 0 AADI
Column: Beta 5520 VVOS 3801 MDXH
Column: Trailingpe 360 AVDX 0 AADI
Column: Forwardpe 271 ARKO 2704 EXAS
Column: Volume 4118 NVDA 678 CAPNR
Column: Marketcap 1413 AAPL 1 AADR
Column: Fiftytwoweeklow 497 BKNG 474 BHST
Column: Fiftytwoweekhigh 764 CETX 1058 DFGP
Column: Pricetosalestrailing12months 15 ABEO 3463 IZTC
Column: Fiftydayaverage 497 BKNG 8 AAPG
Column: Twohundreddayaverage 497 BKNG 8 AAPG
Column: Profitmargins 3363 INBX 3903 MRN

,Symbol,Shortname,Currency,Previousclose,Open,Daylow,Dayhigh,Regularmarketpreviousclose,Regularmarketopen,Regularmarketdaylow,...,Returnonequity,Grossprofits,Freecashflow,Operatingcashflow,Earningsgrowth,Revenuegrowth,Grossmargins,Ebitdamargins,Operatingmargins,Trailingpegratio
0,AADI,"Aadi Bioscience, Inc.",USD,3.14,3.10,3.00,3.10,3.14,3.10,3.00,...,-65.95,-0.03,-0.04,-5.414100e+07,0.0,21.0,-1.10,-252.87,-1.49,0.00
1,AADR,AdvisorShares Dorsey Wright,USD,74.51,72.97,74.25,74.51,74.51,72.97,74.25,...,0.00,0.00,0.00,0.000000e+00,0.0,0.0,0.00,0.00,0.00,0.00
2,AAL,"American Airlines Group, Inc.",USD,16.96,16.80,16.70,17.54,16.96,16.80,16.70,...,0.00,12.89,0.45,2.234000e+09,0.0,1.2,0.24,9.45,0.05,0.21
3,AAME,Atlantic American Corporation,USD,1.51,1.44,1.44,1.55,1.51,1.44,1.44,...,-3.09,0.01,0.01,2.615000e+06,0.0,-3.1,0.08,-0.19,0.00,0.00
4,AAOI,"Applied Optoelectronics, Inc.",USD,30.69,28.58,25.80,29.72,30.69,28.58,25.80,...,-41.44,0.05,-0.04,-4.336000e+07,0.0,4.2,0.26,-23.26,-0.25,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4288,ZVSA,"ZyVersa Therapeutics, Inc.",USD,1.45,1.44,1.37,1.44,1.45,1.44,1.37,...,-152.23,0.00,-0.01,-9.414435e+06,0.0,0.0,0.00,0.00,0.00,0.00
4289,ZYBT,Zhengye Biotechnology Holding L,USD,4.27,4.26,4.23,4.30,4.27,4.26,4.23,...,0.00,0.00,0.00,0.000000e+00,0.0,0.0,0.00,0.00,0.00,0.00
4290,ZYME,Zymeworks Inc.,USD,14.59,14.45,14.45,15.08,14.59,14.45,14.45,...,-28.73,-0.06,-0.01,-5.458200e+07,0.0,-3.1,-1.00,-184.42,-2.14,0.00
4291,ZYXI,"Zynex, Inc.",USD,8.36,8.29,8.15,8.45,8.36,8.29,8.15,...,10.22,0.15,0.02,1.646400e+07,-30.0,0.1,0.80,6.98,0.08,0.00


In [115]:
df.to_csv("stocksData.csv", index=False)
print("Saved data to csv file.")

Saved data to csv file.
